This notebook implements Generative Adversarial Imitation Learning(Ho and Ermon, 2016) on the basic OpenAI Gym environmet of Acrobot-v1.

# Generative Adversarial Imitation Learning (GAIL)
## Introduction
I will make a brief introduction in the main components of reinforcement learning, as well as a short overview of imitation learning to make is more clear where GAIL can be situated and what is the motivation for it. 

Let's start with the basic tool to pose a reinforcement learning setting - the Markov Desicion Process(MDP). A MDP is a four-tuple $(S,A,P(s,a,s'), R(s,a,s'))$, where $S$ is the set of possible states, $A$ the state of possible action, $P(s,a,s')$ is the probability of transition in $s'$ after executing action $a$ when in state $s$, and $R(s,a,s')$ is the reward for this transtion. This way we can describe sequences of interactions with the environment under the use of the Markov property. The goal of the RL agent is to gather as much reward as possible in a long run. To achieve that in a sustainable way, the agent maintains a policy of what action to take when in a certain state. Mostly, we are interested in a stochastic policy - a function $\pi: S \to A$ in the form $\pi(a|s)$. This represents the distribution over actions of what is the right action to do in the current state. Follwoing its goal, the agent aims to learn 

Reinforcement learning problems are formalised as Markov Decision Process (MDP). This framework can fully describe the components and interactions in RL, where the sequence of decisions is chosen with respect to the long-term effect on the total accumulated reward and not to the immediate ones. Formaly, a MDP consists of a n-tuple $\{\mathcal{S}, \mathcal{A}, R, P, \gamma \}$. $\mathcal{S}$ is the set of some discrete states. They are representation of the current environment's state. Based on them, the agent takes decision to act, executing action from the  set of some discrete actions $\mathcal{A}$. The action sets the environment in a new state and responses with reward $R(s)$ for being at this state. P is the tensor of transitions probabilities $P(s,a,s') = p(s_{t+1}=s'|s_t=s, a_t=a)$, called also model of the environment, that fully specifies the outcome of taking action $a$ when in state $s$. When taking actions, the agent considers future rewards if might take along the way. How valuable these future rewards are for the current decision is  given by the discount factor $\gamma$. The policy $\pi(a|s)=Pr\{A_t=a|S_t=s\}$ expresses the preference of the agent for action $a$ when in state $s$. The final goal of the agent is to learn a policy that maximises the expected discounted return $G_t=\sum_i^T \gamma^i R_{t+i+1}$ from the current time step $t$ to the end of the episode at time $T$ (The same is true also for $T \to \infty$). The RL algorithms are focused in the task of finding this optimal policy $\pi^*$.

In imitation learning we might not have a reward function $R$, but additionally we have set of some demonstrations of desired(expert) behaviour, generated from an expert policy $\pi_E$. We need to formalize a optimization task for the agent to be able to learn from the demonstrated behaviour. One way is to do Behaviour cloning, where we directly learn a policy by minimizing the KL divergence with the expert policy:$
\pi^L = argmin_{\hat{\pi}\in\Pi}\mathbb{E}_{q(s,a)}\left[\ln\pi_E(a|s)-\ln\hat{\pi}(a|s)\right]    
$.
Here we seek for the policy $\hat{\pi}$ that is most similar to the expert policy. Learning directly the policy of the demonstrator is sub-optimal: small differences in the environment responses at test time can lead to significant errors - if the agent enters a unknown region, the following behaviour will be sub-optimal. We need some way to introduce ordering over all trajectories in the MDP.

With Inverse Reinforcement Learning (IRL) we try to learn the underlying reward/cost function $c \in C$ that can explain the observations. We assume that the expert has an optimal policy w.r.t. to this function and if uncover it, then the agent can learn the optimal policy and this way be able to become as good as the expert in this task. Because the idea of uses Maximum causal entropy IRL as a starting point, let's look at its objective:

$IRL(\pi_E)=max_{c \in C}(min_{\pi \in \Pi} -H(\pi) + \mathbb{E}_{\pi}[c(s,a)])-\mathbb{E}_{\pi_E}[c(s,a)]$

Maximum causal entropy IRL looks for a cost function $c \in C$ that assigns low cost to the samples from expert policy and high cost to other policies. Here for a given cost function, we try to minimise the expected cost w.r.t. a policy pi, while maintaining the entropy of the policy as high as possible. And then, with the obtained $\pi$, we try to maximise the difference between this term and the expected cost for the expert demonstrations by updating the cost function.

## Motivaion
Inverse Reniforcement Learning learns a cost function that explains the expert behaviour, but we still need to learn a policy after each change of the cost function. This is computationally expensive because we solve the RL problem in a inner loop again and again - for the objective above, we need to solve $RL(c)=argmin_{\pi \in \Pi} -H(\pi) + \mathbb{E}_{\pi}[c(s,a)]$. Then why to learn the cost function explicitly, if this doesn't tell the agent directly what to do? GAIL is one possible solution to learn a policy directly and avoid the intermediate IRL step. 


## The set of cost functions $C$
Now we are looking in the set $C$ of all possible functions $c$ to be sure that the cost function that explains the expert behaviour is among the solutions. If we choose to approximate the cost function with neural network, for example, to be able to cover as much functions as possible, we will need a regulariser to avoid overfitting. The new, regularised objective becomes: 

$IRL_{\psi}(\pi_E)=max_{c \in C} -\psi(c) +(min_{\pi \in \Pi} -H(\pi) + \mathbb{E}_{\pi}[c(s,a)])-\mathbb{E}_{\pi_E}[c(s,a)]$

While the policy can be approximates by any function, also non-convex one like NN, the authors propose the transformation from policies $\pi(a|s)$ to occupancy measures given by $\rho_{\pi}(s,a)= \pi(a|s) \sum_t \gamma^t P(s_t=s|\pi) $. When following a policy $\pi$, this is the distribution of state-action pairs that the agent will encounter. The expected cost for this policy is then given using the occupancy measure by $\mathbb{E}_{\pi}[c(s,a)]=\sum_{s,a} \rho_{\pi}(s,a)c(s,a)$.

Proposition 3.1 in the original paper shows that for $\pi_{\rho}$ is the only policy whose occupancy measure is $\rho$. Then we are allowed to re-write the cenrtal objective as follows:

$IRL_{\psi}(\pi_E)=max_{c \in C} -\psi(c) +(min_{\pi \in \Pi} -H(\pi) + \sum_{s,a} \rho_{\pi}(s,a)c(s,a))-\sum_{s,a} \rho_{\pi_E}(s,a)c(s,a)$


Moreover, Proposition 3.2 in the paper shows that using the convex conjugate of the regulariser with  $\rho_{\pi}-\rho_{\pi_E}$ gives us optimality guarantees that we can find a the optimal policy by optimising the $RL \circ IRL_{\psi}(\pi_E)$ problem directly:
$RL \circ IRL_{\psi}(\pi_E) = argmin_{\pi \in \Pi}-H(\pi)+\psi^*(\rho_{\pi} - \rho_{\pi_E})$ (Derived from the last equation and the definition of convex conjugate). The full proof relies on the fact that 

$L(\rho, c)= -\psi(c) -H(\pi) + \sum_{s,a} \rho_{\pi}(s,a)c(s,a))-\sum_{s,a} \rho_{\pi_E}(s,a)c(s,a)$

has a unique optimum $(\rho^*, c^*)$ for it is true that $c^* \in IRL_{\psi}(\pi_E)$ and $\pi^* \in RL \circ IRL_{\psi}(\pi_E)$ with $\pi_{\rho^*}=\pi^*$

The regularisation in this formulation becomes very important meaning - we can solve the Rl after IRL problem directly, because in this formulation we seek for a policy whose  occupancy measure is close to the expert’s, as measured by the convex function $\psi^*$. 

This observation becomes the central motivation of GAIL. 


## Using different kinds of $\Psi$
As we see, the form of the occupancy measure matching is crucial. If $\psi$ is a constant, meaning no regularisation, the optimisation problem turns to maximising the $-H(\pi)$, subject to $\rho_{\pi}(s,a) = \rho_{\pi_E}(s,a)$, for any state-action pair. This might be a huge space and it just doesn't scale to large environments. Here the cost of a state-action pair serves as dual variables. Of course, the expert behaviour does not include samples of all state-action pairs if the environment  is sufficiently large. This means that we need some other measure of the difference of occupancy measures. 
For this purpose we can use the proposition above, making the convex conjugate $\psi^*$ of the regularisation function the distance measure. For certain seatings of $\psi$, the objective gets the known objective of entropy-regularized apprenticeship learning, where the cost function is for example a linear combination of basis features: if we choose  $\psi(c)=\delta_C(c)$ where $\delta_C(c)=0$ if $c \in C$  and $\infty$ otherwise. This means 

$min_{\pi \in \Pi} -H(\pi) + max_{c \in C} \mathbb{E}_{\pi}[c(s,a)]-\mathbb{E}_{\pi_E}[c(s,a)] = min_{\pi \in \Pi} -H(\pi) + \delta^*_C(\rho_{\pi}-\rho_{\pi_E})$

For this case, there are existing algorithms, alternating between fitting the cost function to maximise the difference of the expected costs of simulated and expert trajectories. And in a second step update the policy with the current cost function. This method practically  scale to large environments. We will see that this is also the main idea behind the GAIL training. But if the true cost function $c^*$ is not in the set $C$ then the algorithm cannot recover the expert behaviour. The reason is given in the equation above - if $c^*$ in not in $C$, then is cannot be recovered by $RL \circ IRL_{\psi}(\pi_E)$, which will then not have an optimal solution for the policy.

This leads to the proposed form of the regulariser psi that at first looks a little scary. First of all, it can adapt to the expert data. But the exact choice of psi comes from the fact that the convex conjugate of psi for the occupancy matching is equal to the optimal negative log loss of a binary classifier that distinguishes between state acton pairs coming from the expert or from the current policy. This is also the connection to Generative adversarial nets, where the Discriminator has to distinguish between expert and generated data and the generator, that tries to generate examples that are indistinguishable for the discriminator from the expert examples.  Now we have the desired effect of avoiding to first learn a cost function, then a policy. As in GAN we gradually improve the examples generated by the policy until they are as good as the expert w.r.t. to a function with high capacity - in this case the discriminator is a neural network. 
 

## The algorithm
I'll not go into detail about trust region policy optimization, but you see here that this is a part of the GAIL learning process. It's simply an elaborated way updating the policy w.r.t. to the reward, as any policy gradient RL algorithm. For a fixed set of expert trajectories, we sample again and again simulated trajectories with the current policy pi. Then we update the discriminator to distinguish between the expert and the simulated samples. Then, using the score that the discriminator assigned to the simulated trajectories as reward, we update the policy parameters with TRPO or PPO. Normally in GAN we continue until the discriminator cannot distinguish between expert and simulated samples.


# Implementation 
## The environment 
I implemented the algorithm and applied it on the open gym Acrobot-v1 environment. I couldn't applied on the previous version, which it is not supported anymore. But there aren't severe differences. The acrobot system includes two joints and two links, where the joint between the two links is controlled by the agent. The goal is to swing the end of the lower link above the line.



GIF


## Get demonstrations

To start the 

In [10]:
gail_obj = GAIL("Acrobot-v1")

Logging data to /tmp/experiments/1627023800/progress.txt

Number of parameters: 	 pi: 11103, 	 v: 10901

Logging data to /tmp/experiments/1627023800/progress.txt

Number of parameters: 	 pi: 11103, 	 v: 10901



In [11]:
gail_obj.train()


###########  Generating trajectories...  #########
Batch 0	 Discriminator: loss: 1.4658763408660889	 expert mean 0.5193204283714294 	 generator mean 0.4826144874095917
###########  Generating trajectories...  #########
Batch 0	 Discriminator: loss: 1.4545559883117676	 expert mean 0.5212961435317993 	 generator mean 0.48978015780448914
------------ Iteration 2 finished! ------------
###########  Generating trajectories...  #########
Batch 0	 Discriminator: loss: 1.448146939277649	 expert mean 0.523190975189209 	 generator mean 0.49444684386253357
------------ Iteration 3 finished! ------------
###########  Generating trajectories...  #########
Batch 0	 Discriminator: loss: 1.4390614032745361	 expert mean 0.5250206589698792 	 generator mean 0.5005791783332825
---------------------------------------
|             EpLen |             495 |
|             Epoch |               0 |
|      AverageEpRet |            -251 |
|          StdEpRet |            5.66 |
|          MaxEpRet |           

/Users/dimitrs/Development/stuttgart-university/imitation_learning/ppo/utils/mpi_tools.py:81: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  x = np.array(x, dtype=np.float32)


------------ Iteration 4 finished! ------------
###########  Generating trajectories...  #########


KeyboardInterrupt: 

In [ ]:
Explain the behaviour of the discriminator with the exploding gradients. Also, explain the vanishing gradients and the changed objecitve 